## Instalando o detectron2


In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import Instances

# Baixando a base de dados


In [ ]:
!wget "https://github.com/psenna/encontrando_gatos/files/7610979/base_dados.zip"
!mkdir base_dados
!unzip -d base_dados/ base_dados.zip 

# Criando a classe para processar os vídeos

In [ ]:
import cv2
import os

In [ ]:
CLASSE_GATOS = 15

In [ ]:
class DetectNet:
    def __init__(self) -> None:
        self.cfg = get_cfg()

        self.cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80  
        
        self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
        self.predictor = DefaultPredictor(self.cfg)
        return

    def processa_video(self, filename, path):
        caminho_arquivo = path + filename
        cap = cv2.VideoCapture(caminho_arquivo)
        occurrences = 0
        while(True):
            ret, frame = cap.read()
            if (frame is None):
                break

            if self.existem_gatos(frame):
                print(f"{filename} possui gatos")
                return

        print(f"{filename} não possui gatos")

    def existem_gatos(self, frame) -> bool:
        deteccoes = self.predictor(frame)
        instancias = deteccoes["instances"].to("cpu")
        instancias = instancias[instancias.pred_classes == CLASSE_GATOS]
        return len(instancias) >= 1

## Rodando o programa

In [ ]:
DIRETORIO = "./base_dados/"

detectnet = DetectNet()

lista_arquivos = os.listdir(DIRETORIO)
total = len(lista_arquivos)

for index, nome_arquivo in enumerate(lista_arquivos):
    print(f"{index+1}/{total} - " + nome_arquivo)
    detectnet.processa_video(nome_arquivo, DIRETORIO)